<a href="https://colab.research.google.com/github/marcossantos08/ProjetosBI/blob/main/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importando biblioteca do Pandas --> Tratar os Dados
import pandas as pd
#Importando a biblioteca de requisições --> Tratar as Requisiçoes da API do IBGE
import requests

In [2]:
def gerar_csv_origem (caminho: str) -> None:
  dados = {
        "id_venda": [1, 2, 3, 4, 5],
        "data_venda": ["2025-12-14", "2025-11-10", "2025-12-12", "2025-11-01","2025-12-15"],
        "valor_venda": [10500.0, 3750.5, 2450.0, 777.0,5100.0],
        "canal": ["Loja", "Online", "Loja", "Online", "Online"],
        "cidade": ["São Paulo", "Rio de Janeiro", "São Paulo", "Sorocaba", "Votora"]
    }
  df = pd.DataFrame(dados)
  df.to_csv(caminho, index=False)


In [3]:
#Extração do CSV
def extracao(caminho: str) -> pd.DataFrame:
  return pd.read_csv(caminho,parse_dates=["data_venda"])

In [4]:
def buscar_municipios_ibge() -> pd.DataFrame:
    url = "https://servicodados.ibge.gov.br/api/v1/localidades/municipios"
    response = requests.get(url)

    if response.status_code != 200:
        raise Exception("Erro ao acessar API de localidades do IBGE")

    data = response.json()

    df_municipios = pd.json_normalize(data)

    df_municipios = df_municipios[
        ["id", "nome", "microrregiao.mesorregiao.UF.sigla"]
    ]

    df_municipios.columns = ["codigo_ibge", "cidade", "uf"]

    df_municipios["cidade"] = df_municipios["cidade"].str.strip().str.title()

    return df_municipios


In [5]:
#Enriquecinhento os dados
def enriquecimento(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Ano-Mês para análise temporal
    df["ano_mes"] = df["data_venda"].dt.to_period("M").astype(str)

    # Flag de canal online
    df["is_online"] = (df["canal"] == "Online").astype("int8")

    # Padronização da cidade tirando os espacos
    df["cidade"] = df["cidade"].str.strip().str.title()

    df_municipios = buscar_municipios_ibge()

    df = df.merge(
        df_municipios,
        on="cidade",
        how="left",
        indicator=True
    )

    df["municipio_encontrado"] = df["_merge"] == "both"

    df["codigo_ibge"] = df["codigo_ibge"].fillna(-1).astype(int)
    df["uf"] = df["uf"].fillna("NA")

    df = df.drop(columns=["_merge"])

    df_municipios_invalidos = df[~df["municipio_encontrado"]]
    df_municipios_validos = df[df["municipio_encontrado"]]



    return df

In [6]:
def carregamento(df: pd.DataFrame, caminho: str) -> None:
    df.to_csv(caminho, index=False)

In [7]:
caminho_origem = "vendasequipamento.csv"
caminho_destino = "vendasequipamento_enriquecido.csv"

gerar_csv_origem(caminho_origem)

df = extracao(caminho_origem)
df_enriquecido = enriquecimento(df)
carregamento(df_enriquecido, caminho_destino)

print("ETL finalizado com sucesso.")
print(f"Arquivo gerado: {caminho_destino}")

ETL finalizado com sucesso.
Arquivo gerado: vendasequipamento_enriquecido.csv
